In [8]:
import sqlite3
import pandas as pd


In [9]:
conn = sqlite3.connect(":memory:")  # Creates a temporary in-memory database
cursor = conn.cursor()


In [10]:
cursor.execute("""
CREATE TABLE Sales (
    sale_id INTEGER PRIMARY KEY,  -- Unique identifier for each sale
    product TEXT,                 -- Name of the product sold
    category TEXT,                -- Category of the product
    quantity INTEGER,             -- Quantity sold
    price REAL,                   -- Price per unit
    sale_date DATE,               -- Date of the sale
    region TEXT                   -- Region where the sale happened
);
""")


In [11]:
sales_data = [
    (1, 'Laptop', 'Electronics', 5, 800, '2024-02-01', 'North'),
    (2, 'Phone', 'Electronics', 10, 500, '2024-02-02', 'South'),
    (3, 'Tablet', 'Electronics', 3, 300, '2024-02-03', 'North'),
    (4, 'Shoes', 'Fashion', 7, 50, '2024-02-04', 'West'),
    (5, 'T-Shirt', 'Fashion', 20, 20, '2024-02-05', 'East'),
    (6, 'Laptop', 'Electronics', 2, 850, '2024-02-06', 'South'),
    (7, 'Phone', 'Electronics', 5, 550, '2024-02-07', 'West'),
    (8, 'Shoes', 'Fashion', 3, 55, '2024-02-08', 'North'),
    (9, 'T-Shirt', 'Fashion', 10, 18, '2024-02-09', 'West'),
    (10, 'Laptop', 'Electronics', 4, 780, '2024-02-10', 'East')
]


In [12]:
cursor.executemany("INSERT INTO Sales VALUES (?, ?, ?, ?, ?, ?, ?)", sales_data)


In [13]:
conn.commit()  # Save changes


In [14]:
query = "SELECT * FROM Sales;"  # Select all rows from the Sales table
df = pd.read_sql_query(query, conn)  # Execute the query and store the result in a DataFrame
df  # Display the data


,sale_id,product,category,quantity,price,sale_date,region
0,1,Laptop,Electronics,5,800.0,2024-02-01,North
1,2,Phone,Electronics,10,500.0,2024-02-02,South
2,3,Tablet,Electronics,3,300.0,2024-02-03,North
3,4,Shoes,Fashion,7,50.0,2024-02-04,West
4,5,T-Shirt,Fashion,20,20.0,2024-02-05,East
5,6,Laptop,Electronics,2,850.0,2024-02-06,South
6,7,Phone,Electronics,5,550.0,2024-02-07,West
7,8,Shoes,Fashion,3,55.0,2024-02-08,North
8,9,T-Shirt,Fashion,10,18.0,2024-02-09,West
9,10,Laptop,Electronics,4,780.0,2024-02-10,East


In [15]:
query = """
SELECT product,
       SUM(quantity) AS total_quantity,
       SUM(quantity * price) AS total_sales
FROM Sales
GROUP BY product;
"""

df = pd.read_sql_query(query, conn)
df  # Display the output


,product,total_quantity,total_sales
0,Laptop,11,8820.0
1,Phone,15,7750.0
2,Shoes,10,515.0
3,T-Shirt,30,580.0
4,Tablet,3,900.0


In [16]:
query = """
SELECT category,
       SUM(quantity * price) AS total_sales
FROM Sales
GROUP BY category
HAVING total_sales > 1000;
"""

df = pd.read_sql_query(query, conn)
df  # Display the output


,category,total_sales
0,Electronics,17470.0
1,Fashion,1095.0


In [17]:
query = """
SELECT product, category, total_sales
FROM (
    SELECT product,
           category,
           SUM(quantity * price) AS total_sales,
           RANK() OVER (PARTITION BY category ORDER BY SUM(quantity * price) DESC) AS rnk
    FROM Sales
    GROUP BY product, category
) AS ranked
WHERE rnk = 1;
"""

df = pd.read_sql_query(query, conn)
df  # Display the output


,product,category,total_sales
0,Laptop,Electronics,8820.0
1,T-Shirt,Fashion,580.0


In [18]:
query = """
SELECT region,
       SUM(quantity * price) AS total_sales
FROM Sales
GROUP BY region
ORDER BY total_sales DESC
LIMIT 3;
"""

df = pd.read_sql_query(query, conn)
df  # Display the output


,region,total_sales
0,South,6700.0
1,North,5065.0
2,East,3520.0
